In [0]:
!pip install -q keras

# Part 1 Data Preprocessing

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Import Dataset

In [0]:
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ANN/Churn_Modelling.csv')
X = dataset.iloc[:,3:13]
Y = dataset.iloc[:,-1]

Create dummy variables

In [0]:
geography = pd.get_dummies(X['Geography'],drop_first=True)
gender = pd.get_dummies(X['Gender'],drop_first=True)

 concatnate the dataframes

In [0]:
X = pd.concat([X,geography,gender],axis=1)

In [0]:
X = X.drop(['Geography','Gender'],axis=1)

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV


In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Embedding,Flatten,LeakyReLU,BatchNormalization,Dropout
from keras.activations import relu,sigmoid

In [0]:
def create_model(layers,activation):
  model = Sequential()
  for i, nodes in enumerate(layers):
    if i == 0 :
      model.add(Dense(nodes,input_dim=X_train.shape[1]))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
    else :
      model.add(Dense(nodes))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
  
  model.add(Dense(units = 1,kernel_initializer='glorot_uniform',activation='sigmoid'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [0]:
model = KerasClassifier(build_fn= create_model, verbose=0)

In [0]:

layers = [(20,),(6,6),(6,6,3)]
activations = ['sigmoid','relu']
param_grid = dict(layers=layers,activation=activations,batch_size=[128,256],epochs=[100])

In [0]:
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

In [59]:
grid_result = grid.fit(X_train, Y_train)

KeyboardInterrupt: ignored

In [0]:
print(grid_result.best_score_,grid_result.best_params_)

0.7959999918937684 {'activation': 'sigmoid', 'batch_size': 128, 'epochs': 30, 'layers': (6,)}


## Part3 Making predictions and evaluating the model

In [0]:
y_pred = grid.predict(X_test)
y_pred = (y_pred > 0.5)

In [0]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred,Y_test)


In [0]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred,Y_test)

In [0]:
print(score)

0.7975
